<a href="https://colab.research.google.com/github/caror42/UNC-Projects/blob/main/rosetteDetection1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
import zipfile
import requests
import cv2
import matplotlib.pyplot as plt
import glob
import random
import os

In [ ]:
# connect colab workspace to drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# check file path and number of images for training and valid data set
!ls -1 /content/drive/MyDrive/MatDat_REU/RosetteData/rosetteAugmentedDataset/train/images | wc -l
!ls -1 /content/drive/MyDrive/MatDat_REU/RosetteData/rosetteAugmentedDataset/valid/images | wc -l

586
28


In [ ]:
# converts bounding boxes in YOLO format to xmin, ymin, xmax, ymax
def yolo2bbox(bboxes):
  xmin, ymin = bboxes[0]-bboxes[2]/2, bboxes[1]-bboxes[3]/2
  xmax, ymax = bboxes[0]+bboxes[2]/2, bboxes[1]+bboxes[3]/2
  return xmin, ymin, xmax, ymax

In [ ]:
# function to plot the bounding boxes to their respective images
def plot_box(image, bboxes, labels):
  h, w, _ = image.shape
  for box_num, box in enumerate(bboxes):
    x1, y1, x2, y2 = yolo2bbox(box)
    # Denormlaize the coordinates
    xmin = int(x1*w)
    ymin = int(y1*h)
    xmax = int(x2*w)
    ymax = int(y2*h)

    thickness = max(2, int(w/275))

    cv2.rectangle(
        image,
        (xmin, ymin), (xmax, ymax),
        color=(0, 0, 255),
        thickness =thickness
    )
  return image

In [ ]:
# function to plot images with the bounding boxes
def plot(image_paths, label_paths, num_samples):
  all_images = []
  all_images.extend(glob.glob(image_paths+'/*.jpg'))
  all_images.extend(glob.glob(image_paths+'/*.JPG'))

  all_images.sort()

  num_images = len(all_images)

  plt.figure(figsize=(15,12))
  for i in range(num_samples):
    j = random.randint(0,num_images-1)
    image_name = all_images[j]
    image_name = '.'.join(image_name.split(os.path.sep)[-1].split('.')[:-1])
    image = cv2.imread(all_images[j])
    with open(os.path.join(label_paths, image_name+'.txt'), 'r') as f:
      bboxes = []
      labels = []
      label_lines = f.readlines()
      for label_line in label_lines:
        label = label_line[0]
        bbox_string = label_line[2:]
        x_c, y_c, w, h = bbox_string.split(' ')
        x_c = float(x_c)
        y_c = float(y_c)
        w = float(w)
        h = float(h)
        bboxes.append([x_c, y_c, w, h])
        labels.append(label)
    result_image = plot_box(image, bboxes, labels)
    plt.subplot(2, 2, i+1)
    plt.imshow(result_image[:, :, ::-1])
    plt.axis('off')
plt.subplots_adjust(wspace=1)
plt.tight_layout()
plt.show()

<Figure size 640x480 with 0 Axes>

In [ ]:
# visualize a few training images
plot(
    image_paths = '/content/drive/MyDrive/Summer_2023/MatDat_REU/RosetteData/rosetteAugmentedDataset/train/images/',
    label_paths= '/content/drive/MyDrive/Summer_2023/MatDat_REU/RosetteData/rosetteAugmentedDataset/train/labels/',
    num_samples = 4,
)

ValueError: ignored

<Figure size 1500x1200 with 0 Axes>

In [ ]:
# generate a yaml file for the complete dataset
%%writefile rosetteAugmented_v8.yaml
path: '/content/drive/MyDrive/MatDat_REU/RosetteData/rosetteAugmentedDataset'
train: 'train/images'
val: 'valid/images'

# class names
names:
  0: 'rosette_object'

Overwriting rosetteAugmented_v8.yaml


In [ ]:
# install the ultralytics package
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.5/605.5 kB 10.2 MB/s eta 0:00:00


In [ ]:
# import YOLO
from ultralytics import YOLO

In [ ]:
# Setting up the hyperparameters
EPOCHS = 50
BATCH = 8
IMG_SIZE = 1280

In [ ]:
# Finetune and train YOLOv8 model.
!yolo \
task = detect \
mode = train \
model = yolov8m.pt \
imgsz = {IMG_SIZE} \
data = /content/rosetteAugmented_v8.yaml \
epochs = {EPOCHS} \
batch = {BATCH} \
name = yolov8m_v8_50e_rosetteAugmented

Ultralytics YOLOv8.0.138 🚀 Python-3.10.6 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/rosetteAugmented_v8.yaml, epochs=50, patience=50, batch=8, imgsz=1280, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8m_v8_50e_rosetteAugmented, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimi

In [ ]:
# evaluate model
!yolo \
task = detect \
mode = val \
model = /content/drive/MyDrive/Summer_2023/MatDat_REU/yolov8m_v8_50e_BrightAug/weights/best.pt \
name = yolov8m_eval \
data = /content/drive/MyDrive/Summer_2023/MatDat_REU/RosetteData/rosetteBrightness_v8.yaml

Ultralytics YOLOv8.0.137 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients
val: Scanning /content/drive/MyDrive/Summer_2023/MatDat_REU/RosetteData/brightnessAugDataset/valid/labels.cache... 64 images, 34 backgrounds, 0 corrupt: 100% 64/64 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 4/4 [00:09<00:00,  2.33s/it]
                   all         64         47      0.619      0.745      0.724      0.496
Speed: 61.4ms preprocess, 68.2ms inference, 0.0ms loss, 3.5ms postprocess per image
Results saved to runs/detect/yolov8m_eval


In [ ]:
# validation image inference
!yolo \
task = detect \
mode = predict \
model = runs/detect/yolov8m_v8_50e_BrightAug/weights/best.pt \
source = /content/drive/MyDrive/Summer_2023/MatDat_REU/RosetteData/rosetteAugmentedDataset/valid/images \
imgsz = 1280 \
name = yolov8m_v8_50e53_infer1280 \
show_labels = True

Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/cfg/__init__.py", line 380, in entrypoint
    model = YOLO(model, task=task)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/engine/model.py", line 106, in __init__
    self._load(model, task)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/engine/model.py", line 155, in _load
    self.model, self.ckpt = attempt_load_one_weight(weights)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py", line 622, in attempt_load_one_weight
    ckpt, weight = torch_safe_load(weight)  # load ckpt
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py", line 561, in torch_safe_load
    return torch.load(file, map_location='cpu'), file  # load
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 791, in load
    with _open_file_like(f, 'rb') as opene

In [ ]:
# function for plotting and displyaing images and predicted annotations
def visualize(result_dir, num_samples=4) :
  plt.figure(figsize=(20,12))
  image_names = glob.glob(os.path.join(result_dir, '*.jpg'))
  random.shuffle(image_names)
  for i, image_name in enumerate(image_names):
    image = plt.imread(image_name)
    plt.subplot(2, 2, i+1)
    plt.imshow(image)
    plt.axis('off')
    if i == num_samples-1:
      break
  plt.tight_layout()
  plt.show()

In [ ]:
# visualize predicted images
visualize('runs/detect/yolov8m_v8_50e53_infer1280/')

NameError: ignored